# Stuff to do
* in final app, give them just the allowed options (dropdown menu, for example)
* Default color options
* Fix BCC being a dick in the z-direction
* Detect cube from orthogonal set of vectors with coefficients -2, -1, 1, 2
* Find "family" of Hamiltonians
* Write to Morten with regards to the due date


# Stuff to talk about
* Setting up the program
* Show of Fermi-surface script
* Go through Kronig-Penney Model (P>0 perhaps)?
* Contract and hand-in date
* Start on thesis writing
* VIP for TA application

In [84]:
from cmp import *
import pdir
%matplotlib qt
#np.seterr(invalid='ignore')
np.set_printoptions(threshold=np.nan, precision=0)

In [91]:
import itertools

def VG_cos(G, V0, a):
    
    x,y = G.T
    VG = np.zeros(G.size // 2)
    
    # We have special values if Gx=0, Gy=0, G=0, G=+/-b1, G=+/-b2
    b1 = eq(y,0) * eq(abs(x), 2 * np.pi / a)
    b2 = eq(x,0) * eq(abs(y), 2 * np.pi / a)
    x0 = eq(x,0) * ~eq(y,0) * ~b1 * ~b2
    y0 = eq(y,0) * ~eq(x,0) * ~b1 * ~b2
    none0 = ~eq(x,0) * ~eq(y,0) * ~b1 * ~b2
    num_x = -2*y[x0]*a**3 * V0 * np.sin(a*y[x0]/2)
    den_x = y[x0]**2 * a**2 - 4*np.pi**2
    num_y = -2*x[y0]*a**3 * V0 * np.sin(a*x[y0]/2)
    den_y = x[y0]**2 * a**2 - 4*np.pi**2
    num_none = -8*V0* np.sin(x[none0]*a/2)*np.sin(y[none0]*a/2)*a**2 * (x[none0]**2*y[none0]**2*a**2 - 2*x[none0]**2*np.pi**2 - 2*y[none0]**2*np.pi**2)
    den_none = (y[none0]**2-4*np.pi**2)*x[none0]*y[none0]*(x[none0]**2-4*np.pi**2)
    VG[x0] = num_x/den_x
    VG[y0] = num_y/den_y
    VG[none0] = num_none/den_none
    VG[b1 + b2] = V0*a**2/2
    VG_mat = np.array([np.roll(VG,i) for i in range(VG.size)])
    return VG_mat

n_k = 101
a = 2
kx = np.linspace(-np.pi / a, np.pi / a, n_k)
ky = np.linspace(-np.pi / a, np.pi / a, n_k)
constants = 1
V0 = 100
lim = 1
num_G = (2*lim+1)**2
# First we create the relevant matrix for some k:
b1 = np.array([2*np.pi/a, 0])
b2 = np.array([0, 2*np.pi/a])
ms = np.array(list(itertools.product(range(-lim, lim + 1), range(-lim, lim + 1))))
recip = np.array([b1,b2])
Gs = ms @ recip
E = np.zeros((num_G, n_k, n_k))
VG_mat = V0 * np.ones((num_G, num_G))
VG_mat = VG_cos(Gs, V0, a)
kxs, kys = np.meshgrid(kx, ky)


for i in range(n_k):
    for j in range(n_k):
        k = np.array([kx[i],ky[j]])
        Diag = constants * np.diag(lattices.mag(k-Gs)**2)
        Full = Diag + VG_mat
        Eigs = np.linalg.eigvalsh(Full)
        E[:,i,j] = Eigs

band = E[0]
fig = plt.figure()
ax = fig.gca(projection="3d")
ax.plot_surface(kxs, kys, band)
plt.show()

In [8]:
def band_finder(x, band=0):
    """
    A function which takes a 1D vector of boolean values and finds the indices of the n'th band of True
    """
    if len(x.shape) != 1:
        print("Only one dimension allowed!")
        return
    
    if not np.array_equal(x, x.astype(bool)):
        print("Only boolean arrays!")
        return
    
    raw_indices = np.arange(x.size)
    
    indices = raw_indices[x]
    
    band_indices = [None, None]
    
    for i in range(band + 1):
        # First we find the beginning of the next band
        beginning_band = np.amin(raw_indices[x])
        band_indices[0] = beginning_band
        
        # Then we kill all before the new band
        whats_left = raw_indices > beginning_band
        raw_indices = raw_indices[whats_left]
        x = x[whats_left]
        
        # Now we can find the end of the band
        end_band = np.amin(raw_indices[~x])
        
        band_indices[1] = end_band
        
        # And kill the current band
        whats_left = raw_indices > end_band
        raw_indices = raw_indices[whats_left]
        x = x[whats_left]
        
    return np.arange(band_indices[0], band_indices[1])

def newtn(x, k, P):
    sin0 = np.sin(x)
    cos0 = np.cos(x)
    next_ = (-P*sin0 + x*(k-cos0))*x/((x**2 + P)*sin0-P*cos0*x)
    return next_

def newtn_calc(x, k, P, iterations = 10):
    for i in range(iterations):
        x -= newtn(x, k, P)
    return x

def fx(x, P):
    return np.cos(x) + P*np.sinc(x/np.pi)

In [9]:
P = 0.1
N = 2000
lims = 6*np.pi
Plot = True

aplot = np.linspace(-lims, lims, N)
f1 = fx(aplot, P)
f2 = fx(aplot, -P)

if Plot:
    fig1 = plt.figure()
    ax1 = plt.subplot(1,2,1)
    ax2 = plt.subplot(1,2,2)
    ax1.plot(aplot, f1)
    ax1.plot(np.array([-lims, lims]), np.array([1,1]))
    ax1.plot(np.array([-lims, lims]), np.array([-1,-1]))
    ax2.plot(aplot, f2)
    ax2.plot(np.array([-lims, lims]), np.array([1,1]))
    ax2.plot(np.array([-lims, lims]), np.array([-1,-1]))
    fig1.show()

In [10]:
# First thing to plot
band_id = 0
N_new = 50
alpha = np.linspace(0, 10*np.pi, N)
# We use numpys sinc function to take care of the pesky problem of alpha=0. Also, we divide the argument by pi as it is the normalized sinc-function
coska = fx(alpha, P)


## Finding the usable bands
above = coska > 1
below = coska < -1
usable = ~above * ~below

## Selecting the band we want
band = band_finder(usable, band = band_id)
alpha_band = alpha[band]
coska_band = coska[band]

## use newtons method to find better energy-limits
limits = np.array([np.amin(alpha_band), np.amax(alpha_band)])
vert_dis = np.array([1, -1])
if band_id % 2 == 1:
    vert_dis *= -1
new_limits = newtn_calc(limits, vert_dis, P, 100)
alpha_new = np.linspace(new_limits[0], new_limits[1], np.floor(N_new/2).astype('int'))
coska_new = (np.cos(alpha_new) + P*np.sinc(alpha_new/np.pi))

# Now we make the whole array:
x_minus = np.flip(-np.arccos(coska_new),0)
x_plus = np.arccos(coska_new)
x = np.hstack((x_minus,x_plus))
z_minus = np.flip(alpha_new**2, 0)
z_plus = alpha_new**2
z = np.hstack((z_minus, z_plus))

xx,yy = np.meshgrid(x,x.T)
zz_0 = np.tile(z,N_new).reshape(N_new,N_new)
zz = zz_0+zz_0.T

if Plot:
    fig = plt.figure()
    ax = fig.gca(projection="3d")
    ax.plot_surface(xx, yy, zz)
    fig.show()

In [11]:
# Inputs
eq = np.isclose
# Lattice vectors (3 vectors of length 3)
a = 1
b = 2
a1 = np.array([1, 0, 0])
a2 = np.array([0, 1, 0])
a3 = np.array([0, 0, 1])
theta = 80*np.pi/180

# Array of basis vectors
basis = np.array([[0,0,0],[0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]])
# Colors for each of the basis vectors
blargh = ('r', 'r','b','b')
# Size multiplier for each of the atoms. Default is 1
sizes = (2,2,1,1)
verbose = True


# Gridline type:
# Soft: Lines along cartesian axes. Takes into account nonequal lattice spacing
# LatticeVectors: Lines along the latticevectors (only on lattice points)
GridType = "lattice"

# Limit type:
# individual: Sets the limits as max(nx*a1,ny*a2,nz*a3), so we include nx unitcells in the a1 direction, etc.
# sum: Sets the limits r_min = n_min*[a1 a2 a3] and likewise for n_max
LimType = "dynamic"
Maxs = [2,2,2]
Mins = [0,0,0]

LatticeType = "conventional fcc"

#Lattice(lattice_name = LatticeType, colors = blargh, sizes = sizes, max_ = Maxs, verbose=True)
#Reciprocal(lattice_name=LatticeType, indices=(1,1,0))

In [12]:
k0 = np.linspace(0,np.pi/2, 100)
x0 = np.cos(k0)
P = 1

def fx(x, k, V0):
    X = np.sqrt(2)*x
    sin0 = np.sin(X)
    cos0 = np.cos(X)
    f = cos0 - V0*sin0/(X) - np.cos(k)
    return f

def fxprime(x, k, V0):
    scale = np.sqrt(2)
    X = scale * x
    sin0 = np.sin(X)
    cos0 = np.cos(X)
    fprime = -scale * sin0 + V0 * scale * sin0 / (2 * x**2) - V0 * cos0/x
    return fprime

def newtn(x, k, P, a=1):
    sin0 = np.sin(x)
    cos0 = np.cos(x)
    next_ = -(cos0*x - np.cos(k*a)*x-sin0)*x/(sin0*x*x + cos0*x - sin0)
    return next_

def newtn_calc(x, k, P, iterations = 10):
    for i in range(iterations):
        x -= newtn(x, k, P)
    return x



#alpha = newtn_calc(x0, k0, 1)

## Deleting NaN
#alpha_NaN = np.isnan(alpha)
#alpha = alpha[~alpha_NaN]
#k = k0[~alpha_NaN]

## Restricting values to below 1.1*pi² and above 0
#usable = alpha < 1.1* np.amax(k0)**2
#above = alpha >= 0
#ids = np.arange(alpha.size)
#non_usable_index=ids[~ (above*usable)]
#least_index = np.amin(non_usable_index)
#usable_index = np.arange(least_index)
#alpha = alpha[usable_index]
#k = k[usable_index]

#fig = plt.figure()
#ax = fig.gca()
#ax.plot(k,alpha)